In [0]:
spark.sql('CREATE SCHEMA IF NOT EXISTS workspace.iris_silver;')

In [0]:
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, TimestampType
from pyspark.sql import functions as F

iris_silver_schema = StructType([
    StructField('sepal_length', DoubleType(), True),
    StructField('sepal_width', DoubleType(), True),
    StructField('petal_length', DoubleType(), True),
    StructField('petal_width', DoubleType(), True),
    StructField('species', StringType(), True),
])

In [0]:
df_raw = spark.table('workspace.iris_raw.df_iris_raw')

display(df_raw)

In [0]:
df_silver = df_raw.drop_duplicates(['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])

df_silver = df_silver.select(
    F.col('sepal_length').cast('double').alias('sepal_length'),
    F.col('sepal_width').cast('double').alias('sepal_width'),
    F.col('petal_length').cast('double').alias('petal_length'),
    F.col('petal_width').cast('double').alias('petal_width'),
    F.col('species').cast('string').alias('species'),
)

assert df_silver.schema == iris_silver_schema

df_silver.write.mode('overwrite').format('delta').saveAsTable('workspace.iris_silver.iris_silver')
